<a href="https://colab.research.google.com/github/rraghavkaushik/Sarvam-Assignment/blob/main/notebooks/einops_rearrange_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [2]:
import re

In [3]:
def parse(pattern: str):
  if '->' not in pattern:
    raise ValueError("Pattern does not ->, make sure that the pattern is right")
  input = pattern.split('->')[0]
  output = pattern.split('->')[1]
  input = find_axes(input)
  output = find_axes(output)
  return input, output

In [4]:
def find_axes(pattern):
    return re.findall(r'\.\.\.|[\w]+|\([^\)]+\)', pattern)

In [5]:
def flatten_axes(axes):
    flat = []
    for ax in axes:
        if ax.startswith("(") and ax.endswith(")"):
            # flat += re.findall(r'\w+', ax)
            flat += re.findall(r'\w+|\.\.\.', ax)
        else:
            flat.append(ax)
    return flat

In [6]:
def transpose(tensor, input, output):
  for i in output:
    if i not in input:
      raise ValueError(f"Output axis '{ax}' not found in input axes")

  perm = [input.index(ax) for ax in output]
  return tensor.transpose(perm)


In [7]:
def is_transpose(pattern) -> bool:
  return ('...' not in pattern) and ('(' not in pattern) and (')' not in pattern)

In [8]:
# def is_split_axes(pattern: str) -> bool:
#   return ( '(' in pattern.split('->')[0] and '(' not in pattern.split('->')[1] )

def split_axes(ip_ax, tensor_shape, axes_lengths, shape):

  # print(ip_ax)
  axes = re.findall(r'\w+', ip_ax)
  print(axes)

  known_ax = [i for i in axes if i in axes_lengths]
  known_dim = [axes_lengths[i] for i in axes if i in axes_lengths]
  print(known_ax, known_dim)

  unknown_ax = [i for i in axes if i not in axes_lengths]
  print(unknown_ax)

  if len(unknown_ax) == 0:
    return axes

  prod = int(np.prod(known_dim))

  if len(unknown_ax) == 1:
    axes_lengths[unknown_ax[0]] = tensor_shape // prod

  sizes = [axes_lengths.get(i) for i in axes]

  if tensor_shape != np.prod(sizes):
    raise ValueError(f'Mismatch in shapes: {tensor_shape} cannot be split into {sizes}')
    # return None

  for i, s in zip(axes, sizes):
    shape[i] = s

  return axes


In [9]:
def merge_axis(op_ax, axes_shape):

  ax_split = re.findall(r'\w+', op_ax)
  return int(np.prod([axes_shape[i] for i in ax_split]))

In [10]:
def repeat_new_axes(tensor, input_axes, output, axes_shape, axes_lengths):
  new_ax = [i for i in output if i not in input_axes]
  # print(new_ax)

  for i in new_ax:
    if i.startswith('(') or i == '...':
      continue
    if i not in axes_lengths:
      raise ValueError(f"The axis {i} needs a specified length")

    tensor = np.expand_dims(tensor, axis=-1)
    tensor = np.repeat(tensor, axes_lengths[i], axis=-1)

    axes_shape[i] = axes_lengths[i]
    input_axes.append(i)

    target_index = output.index(i)
    tensor = np.moveaxis(tensor, -1, target_index)
    input_axes = input_axes[:-1]
    input_axes.insert(target_index, i)

  return tensor, input_axes

In [11]:
def ellipsis(input, tensor_shape, axes):
  # n = len(axes)
  n_ellipsis = len(tensor_shape) - len(axes)

  if n_ellipsis < 0:
    raise ValueError("Not enough dimensions in input tensor")

  ellipsis_dim = tensor_shape[:n_ellipsis]
  ellipsis_ax = [f'_ellipsis_{i}' for i in range(n_ellipsis)]
  return ellipsis_dim, ellipsis_ax, n_ellipsis

In [12]:
def rearrange(tensor: np.array, pattern: str, **axes_lengths) -> np.ndarray:
    input, output = parse(pattern)
    print(input, output)

    # if is_transpose(pattern):
        # print(transpose(tensor, input, output))
        # return transpose(tensor, input, output)

    print("...")
    axes_shape = {}
    input_axes = []
    # output_axes = []
    ellipsis_dim = []
    axes = []

    for ip_ax in input:
        if ip_ax == '...':
          continue
        elif ip_ax.startswith("(") and ip_ax.endswith(")"):
          axes += re.findall(r'\w+', ip_ax)
        else:
          axes.append(ip_ax)
    # print(axes)

    tensor_shape = list(tensor.shape)
    # print(tensor_shape)

    '''ellipsis_dim = []
    ellipsis_ax = []
    n_ellipsis = 0
    if '...' in pattern:
      ellipsis_dim, ellipsis_ax, n_ellipsis = ellipsis(input, tensor_shape, axes)'''
    # else:
    #   ellipsis_dim = []
    #   ellipsis_ax = []
    #   n_ellipsis = 0

    n_ellipsis = len(tensor_shape) - len(axes)
    if '...' in pattern:
      if n_ellipsis < 0:
        raise ValueError("Too many axoes in pattern for input tensor")

    ellipsis_ax = [f'_ellipsis_{i}' for i in range(n_ellipsis)]
    ellipsis_dim = tensor_shape.copy()
    if '...' in pattern:
      for i in axes:
        ind = tensor_shape.index(axes_lengths.get(i, None)) if i in axes_lengths else None
        if ind is not None:
          ellipsis_dim.pop(i)
    # idx = n_ellipsis
    idx = 0
    for ip_ax in input:
        if ip_ax == '...':
          # continue
          # if len( ellipsis_ax) > 0:
          # for ax, dim in zip(ellipsis_ax, ellipsis_dim): # changed ellipses_ax to ellipsis_ax
          #   axes_shape[ax] = dim
          # input_axes.extend(ellipsis_ax) # changed ellipses_ax to ellipsis_ax
          # else:
            # continue
          for i in range(n_ellipsis):
            axes_shape[ellipsis_ax[i]] = tensor_shape[idx]
            input_axes.append(ellipsis_ax[i])
            idx += 1

        elif ip_ax.startswith('(') and ip_ax.endswith(')'):
          ax_split = split_axes(ip_ax, tensor_shape[idx], axes_lengths, axes_shape)
          input_axes.extend(ax_split)
          idx += 1

        else:
          axes_shape[ip_ax] = tensor_shape[idx]
          input_axes.append(ip_ax)
          idx += 1

    # print(axes_shape)
    print("Inferred Input Axes:", input_axes)
    # tensor = tensor.reshape()
    tensor_reshaped = tensor.reshape([axes_shape[i] for i in input_axes])
    # print(tensor_reshaped)

    new_axes_to_repeat = set(output) - set(input_axes)

    if new_axes_to_repeat:
      tensor_reshaped, input_axes = repeat_new_axes(tensor_reshaped, input_axes, output, axes_shape, axes_lengths)

    print('...', input_axes, output)
    print(tensor_reshaped, input_axes)

    output_ax_dim = []
    for op_ax in output:
        if op_ax == '...':
          # continue
          output_ax_dim.extend([axes_shape[i] for i in ellipsis_ax])
        elif op_ax.startswith('(') and op_ax.endswith(')'):
          output_ax_dim.append(merge_axis(op_ax, axes_shape))
        else:
          output_ax_dim.append(axes_shape[op_ax])


    print("final output shape:", output_ax_dim)

    return tensor_reshaped.reshape(output_ax_dim)

In [13]:
# import time

# start = time.time()

x = np.random.rand(2, 12, 4)
result = rearrange(x, 'h w c -> (h w) c')

# end = time.time()

# print(f"time taken is {end - start} seconds")

['h', 'w', 'c'] ['(h w)', 'c']
...
Inferred Input Axes: ['h', 'w', 'c']
... ['h', 'w', 'c'] ['(h w)', 'c']
[[[0.7382343  0.87206454 0.51773856 0.22897942]
  [0.12250103 0.49560876 0.59968373 0.80104725]
  [0.27974644 0.0351886  0.79123105 0.01321814]
  [0.6446844  0.32242723 0.96667481 0.09259652]
  [0.04463949 0.86931434 0.67420125 0.99471282]
  [0.70838359 0.48233255 0.72604661 0.94137465]
  [0.42307554 0.32167025 0.72205579 0.38126462]
  [0.48840298 0.34484622 0.03105307 0.55051006]
  [0.36770113 0.22175605 0.90312967 0.90823864]
  [0.76912325 0.80670623 0.32003111 0.45825427]
  [0.93869867 0.07929451 0.64275462 0.64571757]
  [0.44814424 0.19601208 0.50369513 0.19505048]]

 [[0.79118702 0.14015106 0.64221857 0.90259625]
  [0.02783067 0.36431326 0.88110454 0.52328058]
  [0.17760943 0.54655113 0.59229797 0.63858674]
  [0.57346411 0.281886   0.41104069 0.96921585]
  [0.1936604  0.92178127 0.16505165 0.25177135]
  [0.15281636 0.50372943 0.09872537 0.52653057]
  [0.53662271 0.20494093 0.

'''
for this case:

x = np.random.rand(2, 12, 4)

result = rearrange(x, 'h w c -> w h c', h=3)
'''

With transpose() and without transpose(), the time difference is very minimum.

In [14]:
x = np.random.rand(12, 5)
result = rearrange(x, '(a b) c -> a (b c)', b= 4)


['(a b)', 'c'] ['a', '(b c)']
...
['a', 'b']
['b'] [4]
['a']
Inferred Input Axes: ['a', 'b', 'c']
... ['a', 'b', 'c'] ['a', '(b c)']
[[[1.45877152e-01 3.54618050e-01 3.20361348e-01 2.70602626e-01
   9.67591790e-01]
  [6.61574349e-01 9.69370803e-01 3.83544172e-01 5.86100069e-01
   1.31727001e-01]
  [8.92364356e-01 4.69812306e-01 3.57569073e-01 6.46189701e-01
   4.25724479e-01]
  [7.46754199e-01 4.60951070e-01 6.48829933e-01 1.76650742e-01
   4.61099383e-01]]

 [[3.81658265e-01 5.08737197e-01 9.67514050e-03 3.52051306e-01
   2.34351759e-01]
  [9.25022909e-01 4.22607569e-02 8.64215561e-01 7.97142048e-04
   6.39375677e-01]
  [5.77581805e-02 2.89028544e-01 1.56262049e-01 4.43065222e-01
   8.93210608e-01]
  [1.17020553e-01 1.96612735e-01 1.13013801e-01 8.15310075e-01
   5.51633073e-01]]

 [[2.03100275e-01 9.28715234e-01 9.83275090e-01 7.39908137e-01
   7.98725580e-01]
  [8.59056157e-01 1.76167529e-01 2.44303282e-01 2.04964579e-01
   5.59210524e-01]
  [4.16172781e-01 8.09474675e-01 7.05587762

In [15]:
# Repeat an axis
x = np.random.rand(3, 1, 5)
result = rearrange(x, 'a1 1 c -> a1 b c', b=4)

['a1', '1', 'c'] ['a1', 'b', 'c']
...
Inferred Input Axes: ['a1', '1', 'c']
... ['a1', 'b', '1', 'c'] ['a1', 'b', 'c']
[[[[0.76347382 0.58193829 0.49232591 0.33161157 0.5108093 ]]

  [[0.76347382 0.58193829 0.49232591 0.33161157 0.5108093 ]]

  [[0.76347382 0.58193829 0.49232591 0.33161157 0.5108093 ]]

  [[0.76347382 0.58193829 0.49232591 0.33161157 0.5108093 ]]]


 [[[0.92097682 0.2745932  0.99953687 0.07800244 0.83499547]]

  [[0.92097682 0.2745932  0.99953687 0.07800244 0.83499547]]

  [[0.92097682 0.2745932  0.99953687 0.07800244 0.83499547]]

  [[0.92097682 0.2745932  0.99953687 0.07800244 0.83499547]]]


 [[[0.46847924 0.66216854 0.78897753 0.36355862 0.03787119]]

  [[0.46847924 0.66216854 0.78897753 0.36355862 0.03787119]]

  [[0.46847924 0.66216854 0.78897753 0.36355862 0.03787119]]

  [[0.46847924 0.66216854 0.78897753 0.36355862 0.03787119]]]] ['a1', 'b', '1', 'c']
final output shape: [3, 4, 5]


In [16]:
# Handle batch dimensions
x = np.random.rand(2, 3, 4, 5)
result = rearrange(x, '... h w -> ... (h w)')

['...', 'h', 'w'] ['...', '(h w)']
...
Inferred Input Axes: ['_ellipsis_0', '_ellipsis_1', 'h', 'w']
... ['_ellipsis_0', '_ellipsis_1', 'h', 'w'] ['...', '(h w)']
[[[[0.76208357 0.43241232 0.05325346 0.31173968 0.74551458]
   [0.26212562 0.29532308 0.04675828 0.15664231 0.30992906]
   [0.82123989 0.85339586 0.79155238 0.47094413 0.06078567]
   [0.95530254 0.88242452 0.0970217  0.75145759 0.90993296]]

  [[0.56640712 0.07324555 0.53879965 0.9283309  0.6449143 ]
   [0.72691776 0.59444581 0.0127686  0.68348738 0.90401509]
   [0.06324716 0.01661975 0.55602006 0.98346236 0.02628194]
   [0.9133864  0.82433744 0.05479234 0.12728613 0.58484854]]

  [[0.83300217 0.06130981 0.32794154 0.09398713 0.36504908]
   [0.87905744 0.90151661 0.79628763 0.35052078 0.82037895]
   [0.98686292 0.57368357 0.26387109 0.03519893 0.76820501]
   [0.18128213 0.70204825 0.07175084 0.55246085 0.38539377]]]


 [[[0.57726576 0.99575757 0.82749601 0.54352706 0.83995214]
   [0.03544622 0.41633649 0.81680032 0.48796394 0

In [17]:
# Handle batch dimensions
x = np.random.rand(2, 3, 4, 5)
result = rearrange(x, 'h w ... -> (h w) ...')

['h', 'w', '...'] ['(h w)', '...']
...
Inferred Input Axes: ['h', 'w', '_ellipsis_0', '_ellipsis_1']
... ['h', 'w', '_ellipsis_0', '_ellipsis_1'] ['(h w)', '...']
[[[[0.40472327 0.5515014  0.90380477 0.28291459 0.17385377]
   [0.01726274 0.58467909 0.1420895  0.81730104 0.70519116]
   [0.02061959 0.26183339 0.00638581 0.78856838 0.24586237]
   [0.0554675  0.65083347 0.75625501 0.10384424 0.82821588]]

  [[0.48923618 0.4427283  0.54806555 0.64340433 0.97264033]
   [0.70879021 0.49806426 0.04541766 0.41397011 0.57486109]
   [0.07044741 0.40517493 0.42126921 0.52966682 0.6037577 ]
   [0.27814541 0.14295297 0.73580456 0.98085284 0.54278101]]

  [[0.25985227 0.47165295 0.41589169 0.0406531  0.31338618]
   [0.5657183  0.38793689 0.99823355 0.28065366 0.17229421]
   [0.02778767 0.62975225 0.78584526 0.1921918  0.55628073]
   [0.13655046 0.95670611 0.7849638  0.6108584  0.75347873]]]


 [[[0.12011658 0.95608002 0.2546873  0.51403145 0.7927207 ]
   [0.24664287 0.27161469 0.17827788 0.43890854 0

In [18]:
result.shape

(6, 4, 5)

In [19]:
import torch

In [21]:
x = torch.arange(2 * 3 * 4 * 5 * 6).reshape(2, 3, 4, 5, 6)
y1 = rearrange(x, 'a b c d e -> (a b) c d e')
y1_expected = x.reshape(2 * 3, 4, 5, 6)
assert torch.equal(y1, y1_expected), "Test 1 failed"

['a', 'b', 'c', 'd', 'e'] ['(a b)', 'c', 'd', 'e']
...
Inferred Input Axes: ['a', 'b', 'c', 'd', 'e']
... ['a', 'b', 'c', 'd', 'e'] ['(a b)', 'c', 'd', 'e']
tensor([[[[[  0,   1,   2,   3,   4,   5],
           [  6,   7,   8,   9,  10,  11],
           [ 12,  13,  14,  15,  16,  17],
           [ 18,  19,  20,  21,  22,  23],
           [ 24,  25,  26,  27,  28,  29]],

          [[ 30,  31,  32,  33,  34,  35],
           [ 36,  37,  38,  39,  40,  41],
           [ 42,  43,  44,  45,  46,  47],
           [ 48,  49,  50,  51,  52,  53],
           [ 54,  55,  56,  57,  58,  59]],

          [[ 60,  61,  62,  63,  64,  65],
           [ 66,  67,  68,  69,  70,  71],
           [ 72,  73,  74,  75,  76,  77],
           [ 78,  79,  80,  81,  82,  83],
           [ 84,  85,  86,  87,  88,  89]],

          [[ 90,  91,  92,  93,  94,  95],
           [ 96,  97,  98,  99, 100, 101],
           [102, 103, 104, 105, 106, 107],
           [108, 109, 110, 111, 112, 113],
           [114, 115

In [22]:
y2 = rearrange(x, 'a b c d e -> a b (c d) e')
y2_expected = x.reshape(2, 3, 4 * 5, 6)
assert torch.equal(y2, y2_expected), "Test 2 failed"

['a', 'b', 'c', 'd', 'e'] ['a', 'b', '(c d)', 'e']
...
Inferred Input Axes: ['a', 'b', 'c', 'd', 'e']
... ['a', 'b', 'c', 'd', 'e'] ['a', 'b', '(c d)', 'e']
tensor([[[[[  0,   1,   2,   3,   4,   5],
           [  6,   7,   8,   9,  10,  11],
           [ 12,  13,  14,  15,  16,  17],
           [ 18,  19,  20,  21,  22,  23],
           [ 24,  25,  26,  27,  28,  29]],

          [[ 30,  31,  32,  33,  34,  35],
           [ 36,  37,  38,  39,  40,  41],
           [ 42,  43,  44,  45,  46,  47],
           [ 48,  49,  50,  51,  52,  53],
           [ 54,  55,  56,  57,  58,  59]],

          [[ 60,  61,  62,  63,  64,  65],
           [ 66,  67,  68,  69,  70,  71],
           [ 72,  73,  74,  75,  76,  77],
           [ 78,  79,  80,  81,  82,  83],
           [ 84,  85,  86,  87,  88,  89]],

          [[ 90,  91,  92,  93,  94,  95],
           [ 96,  97,  98,  99, 100, 101],
           [102, 103, 104, 105, 106, 107],
           [108, 109, 110, 111, 112, 113],
           [114, 115

In [23]:
y3 = rearrange(x, 'a b c d e -> a b c d e')
assert torch.equal(y3, x), "Test 3 failed"

['a', 'b', 'c', 'd', 'e'] ['a', 'b', 'c', 'd', 'e']
...
Inferred Input Axes: ['a', 'b', 'c', 'd', 'e']
... ['a', 'b', 'c', 'd', 'e'] ['a', 'b', 'c', 'd', 'e']
tensor([[[[[  0,   1,   2,   3,   4,   5],
           [  6,   7,   8,   9,  10,  11],
           [ 12,  13,  14,  15,  16,  17],
           [ 18,  19,  20,  21,  22,  23],
           [ 24,  25,  26,  27,  28,  29]],

          [[ 30,  31,  32,  33,  34,  35],
           [ 36,  37,  38,  39,  40,  41],
           [ 42,  43,  44,  45,  46,  47],
           [ 48,  49,  50,  51,  52,  53],
           [ 54,  55,  56,  57,  58,  59]],

          [[ 60,  61,  62,  63,  64,  65],
           [ 66,  67,  68,  69,  70,  71],
           [ 72,  73,  74,  75,  76,  77],
           [ 78,  79,  80,  81,  82,  83],
           [ 84,  85,  86,  87,  88,  89]],

          [[ 90,  91,  92,  93,  94,  95],
           [ 96,  97,  98,  99, 100, 101],
           [102, 103, 104, 105, 106, 107],
           [108, 109, 110, 111, 112, 113],
           [114, 1

In [24]:
import torch
from einops import rearrange

def test_equivalent_rearrange_patterns():
    x = torch.arange(2 * 3 * 4 * 5 * 6).reshape(2, 3, 4, 5, 6)

    # Test 1: (a b) c d e
    y1 = rearrange(x, 'a b c d e -> (a b) c d e')
    y1_expected = x.reshape(2 * 3, 4, 5, 6)
    assert torch.equal(y1, y1_expected), "Test 1 failed"

    # Test 2: a b (c d) e
    y2 = rearrange(x, 'a b c d e -> a b (c d) e')
    y2_expected = x.reshape(2, 3, 4 * 5, 6)
    assert torch.equal(y2, y2_expected), "Test 2 failed"

    # Test 3: identity
    y3 = rearrange(x, 'a b c d e -> a b c d e')
    assert torch.equal(y3, x), "Test 3 failed"

    # Test 4: flatten all
    y4 = rearrange(x, 'a b c d e -> (a b c d e)')
    y4_expected = x.reshape(-1)
    assert torch.equal(y4, y4_expected), "Test 4 failed"

    # Test 5: b (c d e) a
    y5 = rearrange(x, 'a b c d e -> b (c d e) a')
    y5_expected = x.permute(1, 2, 3, 4, 0).reshape(3, 4 * 5 * 6, 2)
    assert torch.equal(y5, y5_expected), "Test 5 failed"

    # Test 6: b (a c d) e
    y6 = rearrange(x, 'a b c d e -> b (a c d) e')
    y6_expected = x.permute(1, 0, 2, 3, 4).reshape(3, 2 * 4 * 5, 6)
    assert torch.equal(y6, y6_expected), "Test 6 failed"

    print("All test cases passed!")

test_equivalent_rearrange_patterns()


All test cases passed!


In [35]:
def test():
  x = torch.arange(2 * 3 * 4 * 5 * 6).reshape(2, 3, 4, 5, 6)

  y1 = rearrange(x, "... c d e -> ... (c d) e")
  y1_expected = x.reshape(2, 3, 4 * 5, 6)
  assert torch.equal(y1, y1_expected), "Test 1 failed"

  print(y1.shape, y1_expected.shape)

  y2 = rearrange(x, 'a b c d e -> b (c d e) a')
  y2_ = rearrange(x, "a b ... -> b (...) a")
  assert torch.equal(y2, y2_), "Test 2 failed"

  print(y2.shape, y2_.shape)

  y3 = rearrange(x, "a b c d e -> b (a c d) e")
  y3_ = rearrange(x, "a b ... e -> b (a ...) e")
  assert torch.equal(y3, y3_), "Test 3 failed"

  print(y3.shape, y3_.shape)

In [36]:
test()

torch.Size([2, 3, 20, 6]) torch.Size([2, 3, 20, 6])
torch.Size([3, 120, 2]) torch.Size([3, 120, 2])
torch.Size([3, 40, 6]) torch.Size([3, 40, 6])
